# Import Libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import os
import datetime
from collections import OrderedDict

from pandas_datareader import data as pdr
import fix_yahoo_finance

import statsmodels.api as sm
import scipy.stats as stats
from scipy.spatial import distance
from scipy.linalg import svd
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.covariance import GraphLassoCV

import cvxpy as cp
import random
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Data Pull and Processing

In [ ]:
tickers = [
    'JNJ', 'PFE', 'UNH', 'MRK', 'ABT', # health care
    'XOM', 'CVX', 'COP', 'SLB', 'EOG', # energies
    'BA', 'UNP', 'MMM', 'HON', 'UTX', # industrials
    'BRK-B', 'JPM', 'BAC', 'WFC', 'C', # financial
    'WMT', 'PG', 'KO', 'PEP', 'PM', # consumer staples
    'AMZN', 'HD', 'MCD', 'NKE', 'SBUX', # consumer discretionary 
    'AAPL', 'MSFT', 'V', 'INTC', 'CSCO', # information technologies
    'DWDP', 'ECL', 'APD', 'SHW', 'LYB', # materials / exlude Linde
    'AMT', 'SPG', 'CCI', 'PLD', 'PSA', # real estate
    'GOOGL', 'GOOG', 'VZ', 'T', 'CMCSA', 'DIS', # communication services / exclude Facebook
    'NEE', 'DUK', 'D', 'SO', 'EXC' # utilities
]


In [ ]:
def get_adj_closing_prices(tickers, start_date, end_date):
    all_prices = {}
    for ticker in tickers:
        prices = pdr.get_data_yahoo(ticker, 
            start=start_date,
            end=end_date
        )
        all_prices[ticker] = prices["Adj Close"]
    
    return pd.DataFrame(all_prices)

In [ ]:
prices = get_adj_closing_prices(tickers, datetime.datetime(2010, 12, 31), datetime.datetime(2019, 4, 1))

In [ ]:
prices.head()

In [ ]:
# log returns
daily = np.log(1 + prices.pct_change().dropna())
weekly = np.log(1 + prices.resample('W-FRI').last().pct_change().dropna())
monthly = np.log(1 + prices.resample('BM').last().pct_change().dropna())
yearly = np.log(1 + prices.resample('Y').last().pct_change().dropna())

# % returns
#daily = prices.pct_change().dropna()
#weekly = prices.resample('W-FRI').last().pct_change().dropna()
#monthly = prices.resample('BM').last().pct_change().dropna()
#yearly = prices.resample('Y').last().pct_change().dropna()

In [ ]:
# SP500 annual variance
sp500 = pdr.get_data_yahoo('^GSPC', start=datetime.datetime(2010, 12, 31), end=datetime.datetime(2019, 4, 1))
daily_sp = np.log(1 + sp500["Adj Close"].pct_change().dropna())

for i in range(750,len(daily)):
    print(daily.index[i].strftime('%m/%d/%Y'), np.mean((np.var(daily[(i-252):i])*100)))

In [ ]:
yearly.head()

# PCA

In [ ]:
def normalize(df):
    return (df - df.mean(axis=0)) / df.std(axis=0) # standardize

In [ ]:
# normalize log returns
daily_norm = normalize(daily)
weekly_norm = normalize(weekly)
monthly_norm = normalize(monthly)
yearly_norm = normalize(yearly)

In [ ]:
def component_cumu_weight(data):
    _, s, _ = svd(np.array(data))
    plt.figure(figsize=(6, 4))
    plt.plot(1 + np.arange(s.shape[0]), 100 * np.cumsum(s**2) / np.sum(s**2))
    plt.xlabel('Number of factors')
    plt.ylabel('Explained variance (%)')
    plt.savefig('factor_weight.png', bbox_inches='tight')
    
    return None

In [ ]:
component_cumu_weight(monthly_norm)

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 30)

def plot_pca(data, k): # k is number of most important components
    U, s, Vh = svd(np.array(data))

    x = np.arange(len(data.columns))
    width = 0.15
    
    plt.figure()
    for i in range(k):
        plt.subplot(k, 1, i+1)
        plt.bar(x + (-(k-1)/2 + i) * width, Vh[i,:], width,
            label='Component {} ({:.1f}%)'.format(i, 100 * s[i]**2 / np.sum(s**2)))
        third = daily.columns[Vh[i,:]**2 >= np.percentile(Vh[i,:]**2, 100 * 2 / 3)]
        
        # print('[Component {}] {}'.format(i, ', '.join(list(third))))
        
        plt.xticks(x, data.columns, rotation=90)
        plt.title('Loading vectors')
        plt.legend()
        plt.savefig('pca_loading.png', bbox_inches='tight')
        
        # print(Vh[i,:])
        
    return None

In [ ]:
#plot_pca(daily_norm, 4)

In [ ]:
#plot_pca(weekly_norm, 4)

In [ ]:
#plot_pca(monthly_norm, 4)

In [ ]:
#plot_pca(yearly_norm, 4)

# Sparse Inverse Covariance Matrix

In [ ]:
def plot_inv_cov_mat(data, size):
    # model = GraphLassoCV(alphas=list(np.arange(0.65,2,1)),cv=10,max_iter=5000) # forced sparsity
    model = GraphLassoCV(alphas=10,cv=5,max_iter=5000,mode='cd')
    model.fit(data)
    inv_mat = pd.DataFrame(model.precision_)
    inv_mat.columns = data.columns
    g = model.precision_
    #g = np.zeros((size, size))
    #g[model.precision_>0]=0.5
    #g[model.precision_<0]=-0.5

    plt.figure(figsize=(10, 10))
    plt.imshow(g, interpolation="nearest",
           vmax=1, vmin=-1, 
           cmap=plt.cm.RdBu_r)
    x_ticks = plt.xticks(range(len(data.columns)), data.columns, rotation=90)
    y_ticks = plt.yticks(range(len(data.columns)), data.columns)
    plt.title('Robust Inverse Covariance')
    plt.savefig('inv_full.png', bbox_inches='tight')
    
    return None

In [ ]:
emp_cov = np.cov(np.array(daily_norm.T))
emp_pre = np.linalg.inv(emp_cov)
g = emp_pre
#g = np.zeros((56, 56))
#g[emp_pre>0]=0.5
#g[emp_pre<0]=-0.5

plt.figure(figsize=(10, 10))
plt.imshow(g, interpolation="nearest",
        vmax=1, vmin=-1, 
        cmap=plt.cm.RdBu_r)
x_ticks = plt.xticks(range(len(daily_norm.columns)), daily_norm.columns, rotation=90)
y_ticks = plt.yticks(range(len(daily_norm.columns)), daily_norm.columns)
plt.title('Emp Inverse Covariance')
plt.savefig('inv_emp.png', bbox_inches='tight')

In [ ]:
plot_inv_cov_mat(daily_norm, 56)

In [ ]:
plot_inv_cov_mat(weekly_norm, 56)

In [ ]:
plot_inv_cov_mat(monthly_norm, 56)

In [ ]:
def alpha_selection(data):

    model = GraphLassoCV(alphas=20,max_iter=5000,cv=10)
    model.fit(data)
    plt.figure(figsize=(4, 3))
    plt.semilogx(model.cv_alphas_[0:(len(model.grid_scores_))], 
         np.mean(model.grid_scores_[0:(len(model.grid_scores_))],axis=1),
         'o-')
    plt.axvline(model.alpha_, color='.5')
    plt.title('Model selection')
    plt.ylabel('Cross-validation Score')
    plt.xlabel('alpha')
    plt.savefig('cv_score.png', bbox_inches='tight')
    
    return None

In [ ]:
#alpha_selection(daily_norm)

In [ ]:
#alpha_selection(weekly_norm)

In [ ]:
#alpha_selection(monthly_norm)

# Mean-Variance Computation

In [ ]:
def compute_expected_daily_returns(df):
    return np.array(df.mean())

def compute_daily_covariance_matrix(df):
    return np.cov(np.array(df.T))

In [ ]:
def compute_portfolio_expected_return(return_vector, weights):
    return return_vector.T.dot(weights)

def compute_portfolio_variance(covariance_matrix, weights):
    return weights.T.dot(covariance_matrix).dot(weights)

def compute_portfolio_std(covariance_matrix, weights):
    return np.sqrt(compute_portfolio_variance(covariance_matrix, weights))

In [ ]:
target_return = 0.07

def min_risk_portfolio(expected_returns, covariance_matrix, target_return):
    n = expected_returns.shape[0]

    w = cp.Variable(n)  # Portfolio allocation vector
    ret = expected_returns.T * w
    risk = cp.quad_form(w, covariance_matrix)
    target_ret = cp.Parameter()
    target_ret.value = target_return
    prob = cp.Problem(cp.Minimize(risk), # Restricting to long-only portfolio
                    [ret == target_ret, # match target_return
                    cp.sum(w) == 1, # sum of weights in portfolios sum to 1.
                    w >= 0])
    prob.solve()
    
    if prob.status == 'optimal':
        return w.value
    else:
        return None
    return None

In [ ]:
def efficient_frontier(expected_returns, covariance_matrix):

    min_return = np.min(expected_returns)
    max_return = np.max(expected_returns)
    
    target_returns = np.linspace(min_return, max_return, num=200)
    
    portfolio_weights = []
    
    for tr in target_returns:
        result = min_risk_portfolio(expected_returns, covariance_matrix, tr)
        # only add results if optimization was successful
        if result is not None:
            weights = result
            portfolio_weights.append(weights)

    return portfolio_weights

In [ ]:
def min_variance_portfolio(covariance_matrix, portfolio_weights):
    
    portfolio_stds = []
    
    for i in portfolio_weights:
        portfolio_stds.append(compute_portfolio_std(covariance_matrix, i))
        
    index = np.argmin(portfolio_stds)
    
    return portfolio_weights[index]

In [ ]:
risk_free_rate = 0.026

def max_sharpe_portfolio(expected_returns, covariance_matrix, risk_free_rate, portfolio_weights):
    
    portfolio_stds = []
    returns = []
    for i in portfolio_weights:
        portfolio_stds.append(compute_portfolio_std(covariance_matrix, i))
        returns.append(np.dot(expected_returns, i))
    
    sharpe_ratios = (np.array(returns) - risk_free_rate) / np.array(portfolio_stds)
    index = np.argmax(sharpe_ratios)
    return sharpe_ratios[index], portfolio_weights[index]

In [ ]:
# ************************
# input must be daily data
# ************************

def emp_portfolio_sim(duration, rebal_freq, data):

    returns = []
    turn_over = []

    for i in range(duration-1):
    
        if i != 0:
            last_weights = optimal_weights
    
        end_d = rand_end - (duration - i) * rebal_freq
        start_d = end_d - 252
        
        exp_return = compute_expected_daily_returns(np.exp(data[start_d:end_d]))
        
        annual_return = np.power(exp_return, 252) - 1
   
        cov = compute_daily_covariance_matrix(data[start_d:end_d])
        
        optimal_weights = min_risk_portfolio(annual_return, cov*252, target_return)
        
        #portfolio_weights = efficient_frontier(annual_return, cov*252)
        #sharpe_ratio, optimal_weights = max_sharpe_portfolio(
        #    annual_return, cov*252, risk_free_rate, portfolio_weights)
        
        #portfolio_weights = efficient_frontier(annual_return, cov*252)
        #optimal_weights = min_variance_portfolio(cov*252, portfolio_weights)
    
        real_return = np.exp(np.sum(daily[end_d:(end_d+rebal_freq)]))-1
    
        if i != 0:
            turn_over.append(distance.cityblock(optimal_weights, last_weights))

        returns.append(np.dot(optimal_weights, real_return))

    return returns, turn_over

In [ ]:
# ************************
# input must be daily data
# ************************

def robust_portfolio_sim(duration, rebal_freq, data):

    returns = []
    turn_over = []

    for i in range(duration-1):
    
        if i != 0:
            last_weights = optimal_weights
    
        end_d = rand_end - (duration - i) * rebal_freq
        start_d = end_d - 252
        
        exp_return = compute_expected_daily_returns(np.exp(data[start_d:end_d]))
        
        annual_return = np.power(exp_return, 252) - 1
        
        # sparse cov matrix estimate
        model = GraphLassoCV(alphas=10,cv=5,max_iter=5000,mode='cd')
        model.fit(data[start_d:end_d])
        cov = model.covariance_
        
        optimal_weights = min_risk_portfolio(annual_return, cov*252, target_return)
        
        #portfolio_weights = efficient_frontier(annual_return, cov*252)
        #sharpe_ratio, optimal_weights = max_sharpe_portfolio(
        #    annual_return, cov*252, risk_free_rate, portfolio_weights)
        
        #portfolio_weights = efficient_frontier(annual_return, cov*252)
        #optimal_weights = min_variance_portfolio(cov*252, portfolio_weights)
    
        real_return = np.exp(np.sum(daily[end_d:(end_d+rebal_freq)]))-1
    
        if i != 0:
            turn_over.append(distance.cityblock(optimal_weights, last_weights))

        returns.append(np.dot(optimal_weights, real_return))

    return returns, turn_over

In [ ]:
def print_perf(emp_return, robust_return, emp_turn_over, robust_turn_over):

    total_emp_return = 1
    for i in emp_return:
        total_emp_return *= 1 + i

    total_robust_return = 1
    for i in robust_return:
        total_robust_return *= 1 + i
    
    print("{0:15}".format("Total Return"),
          "{0:10.6f}%".format((total_emp_return-1)*100), "{0:10.6f}%".format((total_robust_return-1)*100))
    print("{0:15}".format("Mean Return"),
          "{0:10.6f}%".format(np.mean(emp_return)*100), "{0:10.6f}%".format(np.mean(robust_return)*100))
    print("{0:15}".format("Return Var"),
          "{0:10.6f}%".format(np.var(emp_return)*100), "{0:10.6f}%".format(np.var(robust_return)*100))    

    print("{0:15}".format("Total TO"),
          "{0:10.6f}%".format(np.sum(emp_turn_over)*100), "{0:10.6f}%".format(np.sum(robust_turn_over)*100))
    print("{0:15}".format("Max TO"),
          "{0:10.6f}%".format(np.max(emp_turn_over)*100), "{0:10.6f}%".format(np.max(robust_turn_over)*100))
    print("{0:15}".format("Mean TO"),
          "{0:10.6f}%".format(np.mean(emp_turn_over)*100), "{0:10.6f}%".format(np.mean(robust_turn_over)*100))
    print("{0:15}".format("TO Var"),
          "{0:10.6f}%".format(np.var(emp_turn_over)*100), "{0:10.6f}%".format(np.var(robust_turn_over)*100))
    
    return (total_emp_return-1)*100, (total_robust_return-1)*100, 
    np.mean(emp_return)*100, np.mean(robust_return)*100, 
    np.var(emp_return)*100, np.var(robust_return)*100, 
    np.sum(emp_turn_over)*100, np.sum(robust_turn_over)*100, 
    np.max(emp_turn_over)*100, np.max(robust_turn_over)*100, 
    np.mean(emp_turn_over)*100, np.mean(robust_turn_over)*100, 
    np.var(emp_turn_over)*100, np.var(robust_turn_over)*100

In [ ]:
duration = 40
rebal_freq = 5
trials = 30

for i in range(trials):
    rand_end = random.randint(1000,len(daily))
    print(daily.index[rand_end].strftime('%m/%d/%Y'))
    emp_return, emp_turn_over = emp_portfolio_sim(duration,rebal_freq,daily)
    robust_return, robust_turn_over = robust_portfolio_sim(duration,rebal_freq,daily)
    print_perf(emp_return, robust_return, emp_turn_over, robust_turn_over)

In [ ]:
duration = 40
rebal_freq = 10
trials = 30

for i in range(trials):
    rand_end = random.randint(1000,len(daily))
    print(daily.index[rand_end].strftime('%m/%d/%Y'))
    emp_return, emp_turn_over = emp_portfolio_sim(duration,rebal_freq,daily)
    robust_return, robust_turn_over = robust_portfolio_sim(duration,rebal_freq,daily)
    print_perf(emp_return, robust_return, emp_turn_over, robust_turn_over)